In [1]:
import numpy as np
import os, cv2
import h5py
import tifffile
from tqdm import tqdm, trange
from deepreg.predict import unwrapped_predict, normalize_batched_image
import tensorflow as tf
import matplotlib.pyplot as plt



batch_size = 8
label_shape = (30, 3)

2025-02-27 22:35:15.847016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 22:35:15.847060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 22:35:15.848311: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Register a video in batches

In [ ]:
tiff_path = "/store1/alicia/jellyfish/concat_cropped_12000-29999.tif"
checkpoint_path = "/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/demo_notebook/long_first_jelly_again/save/ckpt-626"
output_video_path = "/home/brian/data4/brian/PBnJ/out_vids/no_align.mp4"
log_dir = "/home/brian/data4/brian/PBnJ/out_vids/logs"

inputs = tifffile.imread(tiff_path)

In [3]:
os.chdir("/home/brian/notebooks/brian/PBrainAlign_and_Jelly/private_BrainAlignNet/scripts")
from register import set_GPU, register
set_GPU(2)

In [ ]:
#### FIRST WE'RE GOING TO JUST TRY TO ALIGN TO A SINGLE FRAME
## The first batch of good videos was aligned using ckpt-90, going to switch to 626

# output_video_path = "/home/brian/data4/brian/PBnJ/out_vids/first_frame_align.mp4"
output_video_path = "/home/brian/data4/brian/PBnJ/out_vids/psuedo_col_control.mp4"
output_h5_path = "/home/brian/data4/brian/PBnJ/out_vids/first_frame_align.h5"
log_dir = "/home/brian/data4/brian/PBnJ/out_vids/logs"

config_path = os.path.join(checkpoint_path.split("/save")[0], "config.yaml")
model = None

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frames, height, width  = inputs.shape
video = cv2.VideoWriter(output_video_path, fourcc, 20, (width,height), False)

# max_frames = 8
# max_frames = 4000
max_frames = 400
# max_frames = frames

padding = np.array([[0,0],[0,0],[0,0],[1,0]])


# batched_moving_image = normalize_batched_image(inputs[0].astype(np.float32))
# batched_moving_image = np.repeat(batched_moving_image, batch_size, axis=0)
batched_fixed_image = np.repeat(np.expand_dims(inputs[0], axis=0), batch_size, axis=0).astype(np.float32)
batched_fixed_image = np.pad(batched_fixed_image[..., np.newaxis], padding, "constant", constant_values=0)

video.write(inputs[0])

try:
    for frame in trange(1, max_frames, batch_size):
        if frame + batch_size > frames:
            break

        # batched_fixed_image = normalize_batched_image(
        #     inputs[frame:frame+batch_size].astype(np.float32)
        # )
        batched_moving_image = inputs[frame:frame+batch_size].astype(np.float32)
        batched_moving_image = np.pad(batched_moving_image[..., np.newaxis], padding, "constant", constant_values=0)
        
        ddf_output, pred_fixed_image, model = unwrapped_predict(
            batched_fixed_image,
            batched_moving_image,
            log_dir,
            label_shape,
            label_shape,
            model = model,
            model_ckpt_path = checkpoint_path,
            model_config_path = config_path,
        )

        pred_fixed_image = ((pred_fixed_image / np.max(pred_fixed_image)) * 255).astype('uint8')
        # plt.imshow(pred_fixed_image[0][...,1])
        # plt.show()
        for i in range(pred_fixed_image.shape[0]):
            video.write(pred_fixed_image[i][...,1])
        # with h5py.File(output_h5_path, 'w') as f:
        #     for i in range(pred_fixed_image.shape[0]):
        #         f.create_dataset(str(i), data=pred_fixed_image[i])
        # break

    cv2.destroyAllWindows()
    video.release()
except:
    cv2.destroyAllWindows()
    video.release()
    print("INTERRUPTED")

  0%|          | 0/500 [00:00<?, ?it/s]

2025-02-28 17:49:02 | WARNING  | Log directory /home/brian/data4/brian/PBnJ/out_vids/logs exists already.
2025-02-28 17:49:02 | WARNING  | Using customized configuration. The code might break if the config doesn't match the saved model.
Built inputs.
Built control points.
Concatenated images.
{'extract_levels': ListWrapper([0, 1, 2, 3]), 'name': 'local', 'num_channel_initial': 16}
Built backbone.
Built DDF.
Built warping.


100%|██████████| 500/500 [06:54<00:00,  1.21it/s]


In [36]:
import matplotlib
%matplotlib inline

In [ ]:
output_video_path = "/home/brian/data4/brian/PBnJ/out_vids/beginning-no_align.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

images = inputs
# images = images[400:]

frames, height, width  = images.shape

video = cv2.VideoWriter(output_video_path, fourcc, 20, (width,height), False)

for image in trange(max_frames):
    video.write(images[image].astype(np.uint8))

cv2.destroyAllWindows()
video.release()

100%|██████████| 18000/18000 [01:35<00:00, 188.52it/s]
